In [1]:
from sklearn.datasets import fetch_openml
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt


## Load MNIST Dataset:


In [2]:
# Load the MNIST dataset using fetch_openml
mnist = fetch_openml('mnist_784', version=1)

# Extract the features (pixel values) and labels from the dataset
X = mnist.data.values.astype('float32')
y = mnist.target.values.astype('int64')

## Standardize Dataset:

In [4]:
# Define a small epsilon value to add to the standard deviation to avoid division by zero
eps = 1e-8

# Calculate the standard deviation of each feature and replace any zero values with eps
std_dev = np.std(X, axis=0)
std_dev[std_dev == 0] = eps

# Normalize the data by subtracting the mean and dividing by the standard deviation
X = (X - np.mean(X, axis=0)) / std_dev

## Split Dataset:

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test,y_train, y_test = train_test_split(X , y ,test_size=0.2, random_state=42)

## Apply one-hot encoding

In [6]:
# Reshape the input data
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

# Convert y_train and y_test to one-hot encoded vectors
y_train = np.eye(10)[y_train]
y_test = np.eye(10)[y_test]

## Implement Dynamic Neural Network from scratch

In [22]:
import numpy as np

# Sigmoid activation function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Derivative of sigmoid activation function
def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

# Mean Squared Error (MSE) loss function
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# Neural Network class
class NeuralNetwork:
    def __init__(self, x, y ,num_of_layers, size_of_layers):

        num_features = x.shape[1]

        size_of_layers.insert(0, num_features)
        # Size (number of neurons) of each layer 
        self.size_of_layers = size_of_layers
        # Number of layers in the network
        
        num_of_layers = len(size_of_layers) 
        self.num_of_layers = num_of_layers
       
        
        # Initialize random weights between layers
        self.weights = [np.random.randn(size_of_layers[i], size_of_layers[i+1]) for i in range(num_of_layers-1)]

        # Initialize random biases for each layer
        self.biases = [np.random.randn(size_of_layers[i+1]) for i in range(num_of_layers-1)]

        # Store the outputs of each layer
        self.outputs = [np.zeros(size) for size in size_of_layers]

        # Store the errors in each layer (excluding the input layer)
        self.errors = [np.zeros(size) for size in size_of_layers[1:]]
        
    
    def forward(self, x):

        # Set the input layer output as the input data
        self.outputs[0] = x  
        for i in range(self.num_of_layers-1):

            # Compute the weighted sum of inputs and biases
            z = np.dot(self.outputs[i], self.weights[i]) + self.biases[i]

            # Apply the sigmoid activation function
            self.outputs[i+1] = sigmoid(z)  

            # Return the output of the last layer
        return self.outputs[-1]  
    
    def backward(self, x, y_true, learning_rate):
        
        # Perform forward propagation to get the predicted output
        y_pred = self.forward(x)  

        # Compute the error between predicted and true output
        error = y_true - y_pred 

        # Compute the delta (error * derivative of sigmoid) 
        delta = error * sigmoid_derivative(y_pred)

        for i in range(self.num_of_layers-2, -1, -1):
            # Store the error in the current layer
            self.errors[i] = delta 

            # Compute the delta for the previous layer using the weights and derivative of sigmoid
            delta = np.dot(delta, self.weights[i].T) * sigmoid_derivative(self.outputs[i])

            # Compute the gradient of weights using outer product of outputs and errors
            grad_w = np.dot(self.outputs[i].reshape(-1, 1), self.errors[i].reshape(1, -1))

            # Gradient of biases is equal to the errors
            grad_b = self.errors[i]  

            # Update the weights using the learning rate and gradient
            self.weights[i] += learning_rate * grad_w 

            # Update the biases using the learning rate and gradient

            self.biases[i] += learning_rate * grad_b  
    
    def train(self, X_train, y_train, epochs, learning_rate):
        for epoch in range(epochs):
            for x, y in zip(X_train, y_train):

                # Perform backward propagation and update weights and biases for each training example
                self.backward(x, y, learning_rate)
                
            if epoch % 10 == 0:

                # Compute the mean squared error loss for the training set
                train_loss = mse(y_train, self.forward(X_train))
                print(f"Epoch {epoch}: Train Loss = {train_loss:.4f}")

        train_accuracy = self.accuracy(X_train, y_train)
        
        # Compute the training accuracy
        print(f"Train Accuracy: {train_accuracy:.4f}")
    
    def predict(self, X):

        # Perform forward propagation and round the predicted output to the nearest integer (0 or 1)
        return np.round(self.forward(X))
        
        
    def accuracy(self, X, y):
        # Compute the accuracy by comparing predicted output with true output
        y_pred = self.predict(X)
        return np.mean(y_pred == y)
        

def NN(x, y, num_of_layers, size_of_layers):
  
    nn = NeuralNetwork(x ,y ,num_of_layers=num_of_layers, size_of_layers=size_of_layers)
    nn.train(x, y, epochs=100, learning_rate=0.01)
    return nn


### **Build NN with only 2 layers => 1 hidden layer and 1 output layer**

In [23]:
nn1 = NN(X_train, y_train, num_of_layers=2, size_of_layers=[20 , 10])
accuracy1_test = nn1.accuracy(X_test, y_test)


print("Test Accuracy1 :", accuracy1_test)


<ipython-input-22-4dafd82cfc0a>:5: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch 0: Train Loss = 0.0272
Epoch 10: Train Loss = 0.0193
Epoch 20: Train Loss = 0.0185
Epoch 30: Train Loss = 0.0183
Epoch 40: Train Loss = 0.0182
Epoch 50: Train Loss = 0.0180
Epoch 60: Train Loss = 0.0180
Epoch 70: Train Loss = 0.0180
Epoch 80: Train Loss = 0.0180
Epoch 90: Train Loss = 0.0179
Train Accuracy: 0.9774
Test Accuracy1 : 0.9768214285714286


### **Build NN with 3 layers=> 2 hidden layers**

### **Where # of neurons in first layer < # of neurons in second layer and 1 output layer**

In [24]:
nn2 = NN(X_train, y_train, num_of_layers=3, size_of_layers=[20, 30, 10])
accuracy2_test = nn2.accuracy(X_test, y_test)
print("Test Accuracy2 :", accuracy2_test)

<ipython-input-22-4dafd82cfc0a>:5: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch 0: Train Loss = 0.0299
Epoch 10: Train Loss = 0.0199
Epoch 20: Train Loss = 0.0199
Epoch 30: Train Loss = 0.0201
Epoch 40: Train Loss = 0.0203
Epoch 50: Train Loss = 0.0204
Epoch 60: Train Loss = 0.0199
Epoch 70: Train Loss = 0.0198
Epoch 80: Train Loss = 0.0195
Epoch 90: Train Loss = 0.0193
Train Accuracy: 0.9757
Test Accuracy2 : 0.9753214285714286


### **Build NN with 3 layers=> 2 hidden layers**

### **Where # of neurons in first layer > # of neurons in second layer and 1 output layer**

In [25]:
nn3 = NN(X_train, y_train, num_of_layers=3, size_of_layers=[30, 20, 10])
accuracy3_test = nn3.accuracy(X_test, y_test)
print("Test Accuracy3 :", accuracy3_test)

<ipython-input-22-4dafd82cfc0a>:5: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch 0: Train Loss = 0.0325
Epoch 10: Train Loss = 0.0200
Epoch 20: Train Loss = 0.0192
Epoch 30: Train Loss = 0.0193
Epoch 40: Train Loss = 0.0196
Epoch 50: Train Loss = 0.0196
Epoch 60: Train Loss = 0.0195
Epoch 70: Train Loss = 0.0194
Epoch 80: Train Loss = 0.0192
Epoch 90: Train Loss = 0.0192
Train Accuracy: 0.9761
Test Accuracy3 : 0.9757071428571429
